In [1]:
path="/Users/gunnvantsaini/Documents/Work/Machine Learning Course/Python/Module 12 Scrapping/code/html.html"
con=open(path,"r")
html=con.read()
con.close()

In [3]:
print(html)

<!DOCTYPE html>

<html>
    <head>
        <title>GETTING STARTED WITH bs4</title>
    </head>
    <div class="para 1">
        <p>
            This is paragraph one
        </p>
        <p>
            This is paragraph two
        </p>
    </div>
    <div class='para 2'>
        <p>
            This is para 1 in div 2
        </p>
    
    </div>
    <div class="para 1">
        <p>
            This is paragraph three of div with class para 1
        </p>
        <p>
            This is paragraph four of div with class para 1
        </p>
    </div>

</html>


In [4]:
type(html)

str

In [5]:
html[0:12]

'<!DOCTYPE ht'

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup=BeautifulSoup(html,'html.parser')

In [9]:
soup.div

<div class="para 1">
<p>
            This is paragraph one
        </p>
<p>
            This is paragraph two
        </p>
</div>

In [10]:
soup.p

<p>
            This is paragraph one
        </p>

In [11]:
soup.find_all("div")

[<div class="para 1">
 <p>
             This is paragraph one
         </p>
 <p>
             This is paragraph two
         </p>
 </div>,
 <div class="para 2">
 <p>
             This is para 1 in div 2
         </p>
 </div>,
 <div class="para 1">
 <p>
             This is paragraph three of div with class para 1
         </p>
 <p>
             This is paragraph four of div with class para 1
         </p>
 </div>]

In [12]:
soup.find_all("div",class_="para 1")

[<div class="para 1">
 <p>
             This is paragraph one
         </p>
 <p>
             This is paragraph two
         </p>
 </div>,
 <div class="para 1">
 <p>
             This is paragraph three of div with class para 1
         </p>
 <p>
             This is paragraph four of div with class para 1
         </p>
 </div>]

In [13]:
soup.select("div")

[<div class="para 1">
 <p>
             This is paragraph one
         </p>
 <p>
             This is paragraph two
         </p>
 </div>,
 <div class="para 2">
 <p>
             This is para 1 in div 2
         </p>
 </div>,
 <div class="para 1">
 <p>
             This is paragraph three of div with class para 1
         </p>
 <p>
             This is paragraph four of div with class para 1
         </p>
 </div>]

In [19]:
### I want to access a particular element
soup.find_all("div",class_="para 1")[0].find_all("p")[0].text.strip()

'This is paragraph one'

In [28]:
soup.find_all("div",class_="para 1")[1].find_all("p")[0].text.strip()

'This is paragraph three of div with class para 1'

### Simulate a browser
- Do a simple get request to the server where this webpage is hosted
- Run a headless browser using a web-driver (This method)

In [29]:
import requests

In [30]:
url="https://www.goodreads.com/quotes"

In [31]:
html=requests.get(url).text

In [33]:
soup=BeautifulSoup(html,'html.parser')

In [37]:
author=[]
for i in soup.find_all("span",class_="authorOrTitle"):
    author.append(i.text.strip())

In [43]:
## Get all the likes
likes=[]
for i in soup.find_all("a",class_="smallText"):
    likes.append(i.text.strip().split(" ")[0])

In [46]:
likes=[i.text.strip().split(" ")[0] for i in soup.find_all("a",class_="smallText")]

In [55]:
quotes=[]
for i in soup.find_all("div",class_="quoteText"):
    quotes.append(list(i.children)[0].strip())

In [56]:
import pandas as pd

In [59]:
table=pd.DataFrame({'Author':author,'Quote':quotes,'Likes':likes})

In [60]:
def get_table(soup):
    author=[]
    for i in soup.find_all("span",class_="authorOrTitle"):
        author.append(i.text.strip())
    likes=[]
    for i in soup.find_all("a",class_="smallText"):
        likes.append(i.text.strip().split(" ")[0])
    quotes=[]
    for i in soup.find_all("div",class_="quoteText"):
        quotes.append(list(i.children)[0].strip())
    table=pd.DataFrame({'Author':author,'Quote':quotes,'Likes':likes})
    return table

In [62]:
table.shape

(30, 3)

In [63]:
###https://www.goodreads.com/quotes?page=1
###https://www.goodreads.com/quotes?page=2

In [64]:
stub="https://www.goodreads.com/quotes?page={}"

In [68]:
stub.format(30)

'https://www.goodreads.com/quotes?page=30'

In [69]:
from tqdm import tqdm

In [70]:
T=[]
for i in tqdm(range(1,30)):
    url=stub.format(i)
    html=requests.get(url).text
    soup=BeautifulSoup(html,'html.parser')
    T.append(get_table(soup))

100%|██████████| 29/29 [01:37<00:00,  3.38s/it]


In [71]:
big_table=pd.concat(T,axis=0)

In [72]:
big_table.shape

(870, 3)

In [73]:
big_table.head(2)

,Author,Quote,Likes
0,Marilyn Monroe,"“I'm selfish, impatient and a little insecure....",150443
1,Oscar Wilde,“Be yourself; everyone else is already taken.”,148900


In [74]:
url="https://www.goodreads.com/list/show/19.Best_for_Book_Clubs"

In [75]:
html=requests.get(url).text

In [76]:
soup=BeautifulSoup(html,'html.parser')

In [80]:
### name
name=[]
for i in soup.find_all("a",class_="bookTitle"):
    name.append(i.span.text.strip())

In [84]:
author=[]
for i in soup.find_all("div",class_="authorName__container"):
    author.append(i.a.span.text.strip())

In [87]:
import re

In [88]:
pattern=re.compile(r'\d\.\d{2}')

In [93]:
ratings=[]
for i in soup.find_all("span",class_="minirating"):
    ratings.append(re.findall(pattern,i.text)[0])

In [95]:
table=pd.DataFrame({'name':name,'author':author,'ratings':ratings})

In [98]:
table.shape

(100, 3)

In [97]:
##https://www.goodreads.com/list/show/19.Best_for_Book_Clubs?page=2
##https://www.goodreads.com/list/show/19.Best_for_Book_Clubs?page=3

In [99]:
def get_table(soup):
    name=[]
    for i in soup.find_all("a",class_="bookTitle"):
        name.append(i.span.text.strip())
    author=[]
    for i in soup.find_all("div",class_="authorName__container"):
        author.append(i.a.span.text.strip())
    ratings=[]
    for i in soup.find_all("span",class_="minirating"):
        ratings.append(re.findall(pattern,i.text)[0])
    table=pd.DataFrame({'name':name,'author':author,'ratings':ratings})
    return table

In [100]:
'''T=[]
for i in range(1,10):
    url=gen_url()
    html=requests.get(url).text
    soup=BeautifulSoup(html,'html.parser')
    T.append(get_table(soup))'''

"T=[]\nfor i in range(1,10):\n    url=gen_url()\n    html=requests.get(url).text\n    soup=BeautifulSoup(html,'html.parser')\n    T.append(get_table(soup))"

In [101]:
url="https://www.naukri.com/data-science-jobs?k=data%20science"

In [102]:
html=requests.get(url).text

In [103]:
print(html)

<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;naukri&#46;com&#47;data&#45;science&#45;jobs&#63;" on this server.<P>
Reference&#32;&#35;18&#46;e3a21ab8&#46;1586149545&#46;3d6b376a
</BODY>
</HTML>



In [104]:
#!pip install selenium

In [106]:
from selenium import webdriver
import os

In [107]:
#os.chdir(path to the unzipped file of the executables)
browser=webdriver.Firefox()

In [108]:
browser.get(url)

In [109]:
html=browser.page_source

In [111]:
soup=BeautifulSoup(html,'html.parser')

In [ ]:
#https://www.naukri.com/data-science-jobs-2?k=data%20science
#https://www.naukri.com/data-science-jobs-3?k=data%20science